sbpy.data.Ephem Example Notebooks
=================================

[Ephem](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.Ephem.html#sbpy.data.Ephem) provides functionality to query, calculate, manipulate, and store ephemerides and observational information.

Querying Asteroid Ephemerides from JPL Horizons
---------------------------------------

Query ephemerides for three asteroids in a given date range as observed from Maunakea (observatory code 568) using [sbpy.data.Ephem.from_horizons](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.Ephem.html#sbpy.data.Ephem.from_horizons). This function uses [astroquery.jplhorizons](https://astroquery.readthedocs.io/en/latest/jplhorizons/jplhorizons.html) in the background and creates Ephem objects from the query results:

In [1]:
from sbpy.data import Ephem
import astropy.units as u
from astropy.time import Time

targets = ['2100', '2018 RC1', 'Ganymed']
eph = Ephem.from_horizons(targets, location=568, 
                          epochs={'start': Time('2018-01-01'), 
                                  'stop': Time('2018-02-01'), 
                                  'step':1*u.hour})

eph.table

targetname,H,G,solar_presence,flags,RA,DEC,RA_app,DEC_app,RA*cos(Dec)_rate,DEC_rate,AZ,EL,AZ_rate,EL_rate,sat_X,sat_Y,sat_PANG,siderealtime,airmass,magextinct,V,surfbright,illumination,illum_defect,sat_sep,sat_vis,ang_width,PDObsLon,PDObsLat,PDSunLon,PDSunLat,SubSol_ang,SubSol_dist,NPole_ang,NPole_dist,EclLon,EclLat,r,r_rate,delta,delta_rate,lighttime,vel_sun,vel_obs,elong,elongFlag,alpha,lunar_elong,lunar_illum,sat_alpha,sunTargetPA,velocityPA,OrbPlaneAng,constellation,TDB-UT,ObsEclLon,ObsEclLat,NPole_RA,NPole_DEC,GlxLon,GlxLat,solartime,earth_lighttime,RA_3sigma,DEC_3sigma,SMAA_3sigma,SMIA_3sigma,Theta_3sigma,Area_3sigma,RSS_3sigma,r_3sigma,r_rate_3sigma,SBand_3sigma,XBand_3sigma,DoppDelay_3sigma,true_anom,hour_angle,alpha_true,PABLon,PABLat,epoch
,mag,,,,deg,deg,deg,deg,arcsec / h,arcsec / h,deg,deg,arcsec / min,arcsec / min,arcsec,arcsec,deg,h,,mag,mag,mag / arcsec2,%,arcsec,arcsec,,arcsec,deg,deg,deg,deg,deg,arcsec,deg,arcsec,deg,deg,AU,km / s,AU,km / s,min,km / s,km / s,deg,,deg,deg,%,deg,deg,deg,deg,,s,deg,deg,deg,deg,deg,deg,,min,arcsec,arcsec,arcsec,arcsec,deg,arcsec2,arcsec,km,km / s,Hz,Hz,s,deg,,deg,deg,deg,
str24,float64,float64,str1,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,float64,float64,float64,float64,float64,float64,float64,str3,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object
2100 Ra-Shalom (1978 RA),16.05,0.12,*,,298.39175,-15.55282,298.63771,-15.5047,134.878,14.838,190.6996,54.0982,853.05,-156.97,57021.64,27057.68,68.218,20.3416632849,1.233,0.134,18.53,4.31,97.902,3.4e-05,64368.56,*,0.001631,0.0,0.0,0.0,0.0,242.49,0.0,0.0,0.0,313.5825,9.6866,1.052945357469,8.940962,1.94452400046837,11.5251474,16.172113,24.87625,54.39145,17.8802,/T,16.6621,147.9,98.1,145.4537,62.577,267.646,7.52998,Sgr,69.18394,297.6297492,5.226016,0.0,0.0,25.501757,-20.596365,13.5798077532,0.000355,0.023,0.009,0.024,0.004,-18.146,0.0,0.025,15.8015,7.2e-06,0.04,0.15,0.000105,145.6446,0.432482349,16.657,305.4411,7.5319,2458119.5
2100 Ra-Shalom (1978 RA),16.05,0.12,*,,298.43064,-15.54874,298.67659,-15.50057,134.91,14.93554,212.434,48.8381,739.13,-453.94,57001.51,27060.06,68.206,21.3444013036,1.327,0.145,18.53,4.31,97.904,3.4e-05,64350.73,*,0.001631,0.0,0.0,0.0,0.0,242.48,0.0,0.0,0.0,313.6126,9.68,1.053160425562,8.9340229,1.94480259652141,11.6279921,16.17443,24.86934,54.43705,17.8752,/T,16.6541,148.6,98.3,145.4667,62.569,267.641,7.5269,Sgr,69.183941,297.6674688,5.2228937,0.0,0.0,25.521794,-20.629056,14.5795191048,0.000355,0.023,0.009,0.024,0.004,-18.153,0.0,0.025,15.7967,7.2e-06,0.04,0.15,0.000105,145.675,1.432628363,16.6489,305.475,7.5269,2458119.541666667
2100 Ra-Shalom (1978 RA),16.05,0.12,*,,298.46956,-15.54464,298.71549,-15.49642,135.0128,15.02554,228.1056,39.6563,596.08,-630.15,56981.11,27062.31,68.195,22.347139323,1.564,0.17,18.53,4.31,97.906,3.4e-05,64332.6,*,0.00163,0.0,0.0,0.0,0.0,242.47,0.0,0.0,0.0,313.6427,9.6734,1.053375326608,8.927084,1.94508355503987,11.7203265,16.176767,24.86243,54.50724,17.8702,/T,16.6461,149.4,98.5,145.4798,62.561,267.636,7.52381,Sgr,69.183941,297.7052061,5.219786,0.0,0.0,25.541865,-20.661751,15.5792237152,0.000355,0.023,0.009,0.024,0.004,-18.159,0.0,0.025,15.7918,7.2e-06,0.04,0.15,0.000105,145.7054,2.432773489,16.6409,305.509,7.522,2458119.583333333
2100 Ra-Shalom (1978 RA),16.05,0.12,*,m,298.50851,-15.54051,298.75441,-15.49223,135.1767,15.10336,238.9077,28.2827,476.25,-724.98,56960.39,27064.45,68.183,23.349877343,2.102,0.229,18.53,4.31,97.908,3.4e-05,64314.12,*,0.00163,0.0,0.0,0.0,0.0,242.46,0.0,0.0,0.0,313.6727,9.6669,1.053590060624,8.9201455,1.94536654232

The resulting `Ephem` object provides the same functionality as all other `sbpy.data` objects; for instance, you can easily convert values to other units:

In [2]:
import numpy as np
import astropy.units as u

eph['RA'][0].to('arcsec')

<Quantity 1074210.3 arcsec>

Querying Satellite Ephemerides from JPL Horizons (and other ambiguous queries)
------------------------------------------------

Query ephemerides for satellite `'GOES-1'` for a number of epochs as observed from Magdalena Ridge Observatory (observatory code `'H01'`):

In [3]:
eph = Ephem.from_horizons('GOES-1', epochs=Time([2453452.123245, 2453453.34342, 2453454.342342], format='jd'))

QueryError: Error raised by astroquery.jplhorizons: Unknown target (GOES-1). Maybe try different id_type?
The following query was attempted: https://ssd.jpl.nasa.gov/horizons_batch.cgi?batch=1&TABLE_TYPE=OBSERVER&QUANTITIES=%271%2C2%2C3%2C4%2C5%2C6%2C7%2C8%2C9%2C10%2C11%2C12%2C13%2C14%2C15%2C16%2C17%2C18%2C19%2C20%2C21%2C22%2C23%2C24%2C25%2C26%2C27%2C28%2C29%2C30%2C31%2C32%2C33%2C34%2C35%2C36%2C37%2C38%2C39%2C40%2C41%2C42%2C43%27&COMMAND=%22GOES-1%3B%22&SOLAR_ELONG=%220%2C180%22&LHA_CUTOFF=0&CSV_FORMAT=YES&CAL_FORMAT=BOTH&ANG_FORMAT=DEG&APPARENT=AIRLESS&REF_SYSTEM=J2000&EXTRA_PREC=NO&CENTER=%27500%27&TLIST=2453452.123245%0A2453453.34342%0A2453454.342342&SKIP_DAYLT=NO

This function call raises a `QueryError`. The reason for this error is that astroquery.jplhorizons assumes that the provided target string is a small body. However, GOES-1 is an artifical satellite. As suggested in the provided error message, we need to specificy an `id_type` that is different from the default value (`'smallbody'`). Since sbpy.data.Ephem.from_horizons builds upon astroquery.jplhorizons, we can provide optional parameters for the latter directly to the former. According to the [JPL Horizons documentation](https://ssd.jpl.nasa.gov/?horizons_doc#selection), spacecraft are grouped together with major body, so let's try:

In [4]:
eph = Ephem.from_horizons('GOES-1', id_type='majorbody',
                          epochs=Time([2453452.123245, 2453453.34342, 2453454.342342], format='jd'))

QueryError: Error raised by astroquery.jplhorizons: Ambiguous target name; provide unique id:
  ID#      Name                               Designation  IAU/aliases/other   
  -------  ---------------------------------- -----------  ------------------- 
  -108366  GOES-1 (spacecraft)                                                  
  -124786  GOES-10 (spacecraft)                                                 
  -126352  GOES-11 (spacecraft)                                                 
  -126871  GOES-12 (spacecraft)                                                 
  -135491  GOES-14 (spacecraft)                                                 
 
The following query was attempted: https://ssd.jpl.nasa.gov/horizons_batch.cgi?batch=1&TABLE_TYPE=OBSERVER&QUANTITIES=%271%2C2%2C3%2C4%2C5%2C6%2C7%2C8%2C9%2C10%2C11%2C12%2C13%2C14%2C15%2C16%2C17%2C18%2C19%2C20%2C21%2C22%2C23%2C24%2C25%2C26%2C27%2C28%2C29%2C30%2C31%2C32%2C33%2C34%2C35%2C36%2C37%2C38%2C39%2C40%2C41%2C42%2C43%27&COMMAND=%22

Another error. This time, the error refers to the target name being ambiguous and provides a list of objects in the database that match the search string. We pick `GOES-1` and use the unique id provided for this object (note that in this case we have to use `id_type='id'`):

In [5]:
eph = Ephem.from_horizons('-108366', id_type='id',
                          epochs=Time([2453452.123245, 2453453.34342, 2453454.342342], format='jd'))
eph.table

targetname,solar_presence,flags,RA,DEC,RA_app,DEC_app,RA*cos(Dec)_rate,DEC_rate,AZ,EL,AZ_rate,EL_rate,sat_X,sat_Y,sat_PANG,siderealtime,airmass,magextinct,V,surfbright,illumination,illum_defect,sat_sep,sat_vis,ang_width,PDObsLon,PDObsLat,PDSunLon,PDSunLat,SubSol_ang,SubSol_dist,NPole_ang,NPole_dist,EclLon,EclLat,r,r_rate,delta,delta_rate,lighttime,vel_sun,vel_obs,elong,elongFlag,alpha,lunar_elong,lunar_illum,sat_alpha,sunTargetPA,velocityPA,OrbPlaneAng,constellation,TDB-UT,ObsEclLon,ObsEclLat,NPole_RA,NPole_DEC,GlxLon,GlxLat,solartime,earth_lighttime,RA_3sigma,DEC_3sigma,SMAA_3sigma,SMIA_3sigma,Theta_3sigma,Area_3sigma,RSS_3sigma,r_3sigma,r_rate_3sigma,SBand_3sigma,XBand_3sigma,DoppDelay_3sigma,true_anom,hour_angle,alpha_true,PABLon,PABLat,epoch
,,,deg,deg,deg,deg,arcsec / h,arcsec / h,deg,deg,arcsec / min,arcsec / min,arcsec,arcsec,deg,h,,mag,mag,mag / arcsec2,%,arcsec,arcsec,,arcsec,deg,deg,deg,deg,deg,arcsec,deg,arcsec,deg,deg,AU,km / s,AU,km / s,min,km / s,km / s,deg,,deg,deg,%,deg,deg,deg,deg,,s,deg,deg,deg,deg,deg,deg,,min,arcsec,arcsec,arcsec,arcsec,deg,arcsec2,arcsec,km,km / s,Hz,Hz,s,deg,,deg,deg,deg,
str29,str1,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,float64,float64,float64,float64,float64,float64,float64,str3,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,object
GOES-1 (spacecraft) (-108366),,,292.53054,-13.56267,292.60033,-13.55515,53839.88,5312.911,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,999,0.0,0.0,0.0,33.011,0.0,0.0,--,0.0,0.0,0.0,0.0,0.0,84.09,0.0,0.0,0.0,182.0342,0.0022,0.996462370793,-2.4006489,0.00028198033874,7.54e-05,0.002345,28.94338,3.07344,70.1227,/L,109.8618,145.1,89.7,0.0,264.115,52.735,-8.48354,Sgr,64.185623,292.173953,8.1630192,0.0,0.0,24.955623,-14.610802,--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,--,109.8638,236.6495,7.1011,2453452.123245
GOES-1 (spacecraft) (-108366),,,11.72219,3.09356,11.78241,3.1196,52437.73,13395.67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,999,0.0,0.0,0.0,0.608,0.0,0.0,--,0.0,0.0,0.0,0.0,0.0,258.51,0.0,0.0,0.0,183.2266,-0.0005,0.996620933601,0.9627722,0.00028194287078,-0.0005526,0.002345,26.88044,3.07419,8.9376,/T,171.0605,147.7,95.8,0.0,78.509,65.731,1.94249,Psc,64.185629,12.0529652,-1.7868197,0.0,0.0,120.666256,-59.760301,--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,--,171.055,97.7863,-11.5368,2453453.34342
GOES-1 (spacecraft) (-108366),,,12.1502,3.20556,12.21043,3.23158,52443.95,13372.85,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,999,0.0,0.0,0.0,0.539,0.0,0.0,--,0.0,0.0,0.0,0.0,0.0,259.73,0.0,0.0,0.0,184.2166,-0.0005,0.996902159044,0.9379062,0.00028194166434,-0.0005518,0.002345,26.8658,3.0742,8.4107,/T,171.5875,159.7,98.8,0.0,79.723,65.779,1.99614,Psc,64.185633,12.4904318,-1.8500582,0.0,0.0,121.519585,-59.660567,--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,--,171.582,98.5263,-12.7094,2453454.342342


Using Ephem objects
-------------------

... please refer to [this notebook](https://github.com/NASA-Planetary-Science/sbpy-tutorial/blob/master/notebooks/data/General_concepts.ipynb) for some examples.

Computing Ephemerides with OpenOrb
-------------------------------

`sbpy.data.Ephem.from_oo` provides a way to compute ephemerides from an [sbpy.data.Orbit](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.Orbit.html#sbpy.data.Orbit) object. This function requires [pyoorb](https://github.com/oorb/oorb/tree/master/python) to be installed.

However, you can use the following code snippet on your computer locally to compute ephemerides for a number of orbits that were obtained from the JPL Horizons system as observed from he Discovery Channel Telescope (observatory code `'G37'`):

In [6]:
from sbpy.data import Ephem, Orbit
from astropy.time import Time
from numpy import arange

orbits = Orbit.from_horizons(['12893', '3552', '2018 RC3'])
epochs = Time(Time.now().jd + arange(0, 3, 1/24), format='jd')
eph = Ephem.from_oo(orbits, epochs, location='G37')

eph.table

targetname,RA,DEC,RA*cos(Dec)_rate,DEC_rate,alpha,elong,r,Delta,V,pa,TopEclLon,TopEclLat,OppTopEclLon,OppTopEclLat,HelEclLon,HelEclLat,OppHelEclLon,OppHelEclLat,EL,ELsun,ELmoon,lunarphase,lunarelong,x,y,z,vx,vy,vz,obsx,obsy,obsz,trueanom,epoch
,deg,deg,deg / d,deg / d,deg,deg,AU,AU,mag,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,,deg,AU,AU,AU,AU / d,AU / d,AU / d,AU,AU,AU,deg,
str26,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object
12893 Mommert (1998 QS55),160.5605433652289,7.616803275597487,0.3377250233917056,-0.13114809656147242,8.58444791876962,26.299483859514886,3.0110669407671287,3.8867522803098455,19.844162969653635,111.22258736329616,159.1823949140605,-0.5504386612061156,-153.70581022224005,-0.5530505141086918,167.7658849561184,-0.709645678046231,-145.12232018018213,-0.7122575309488072,62.68275172452956,64.17299555206097,34.41822991913667,0.30234478433781764,40.65052553931442,-2.9424595839708667,0.6380164141830137,-0.03729307860394834,-0.002299004878260299,-0.00930003608787433,0.00036758983507743714,0.6903845189796334,-0.7432489993656337,4.624281790027349e-05,157.99809421810178,2458701.372966158
12893 Mommert (1998 QS55),160.5747408896238,7.61133603868933,0.3377942293565847,-0.13128235365768479,8.576944876175885,26.275084958661527,3.011078250171949,3.8870117866015326,19.844009374134046,111.2384128796202,159.19749120691597,-0.5502275967457549,-153.73020647157435,-0.5530366243997615,167.77347757167203,-0.7093515515199836,-145.1542201068183,-0.7121605791739903,61.15171282896108,53.2959480152309,43.34650569090973,0.3055411972370575,41.064799756306805,-2.942555344707169,0.6376289206209875,-0.03727776256100396,-0.0022976758235600803,-0.00930032400135261,0.00036760667350933893,0.6908952744686983,-0.7427714709186821,4.9733694451677885e-05,158.0056921893977,2458701.4146328247
12893 Mommert (1998 QS55),160.5889457732859,7.6058633052903994,0.338081994948384,-0.131410706773356,8.569432758152319,26.25060935209744,3.011089555856701,3.8872733974067195,19.843856438386226,111.24084428914672,159.21259648236662,-0.550018289504123,-153.75467930992767,-0.5530322930361831,167.78107012697208,-0.7090574148316426,-145.18620566532223,-0.7120714183637028,53.94842969102581,41.292456736895836,49.529544603741776,0.3085406047904319,41.45336357945285,-2.9426510500407788,0.637241415175323,-0.03726244582092493,-0.0022963467409503613,-0.009300611736533699,0.00036762350482511663,0.6914080643124827,-0.7422939073386658,5.3362772246129214e-05,158.01329010372135,2458701.4562994917
12893 Mommert (1998 QS55),160.60316686657896,7.600385473313785,0.33856988610256816,-0.131525978658383,8.561903654356035,26.226042777321478,3.011100857821261,3.8875369164860833,19.843703728039117,111.22990999483427,159.22771872350228,-0.5498070783730964,-153.77924323764037,-0.5530197230614619,167.78866262207154,-0.7087632679846287,-145.21829933907108,-0.7119759126729943,43.697153614843124,28.965175771955643,51.535221960518285,0.31143545244875936,41.82825899726379,-2.9427466999709813,0.6368538978517629,-0.03724712838394105,-0.0022950176304461023,-0.009300899293418824,0.0003676403290247244,0.6919228994659794,-0.7418157316547265,5.687977306976913e-05,158.02088796092224,2458701.497966158
12893 Mommert (1998 QS55),160.61741195145785,7.594903222274468,0.33922578266180586,-0.1316218962579288,8.554350739544844,26.201377221409434,3.011112156066126,3.8878020012497414,19.843550770868028,111.20658968203007,159.24286482594192,-0.5495904373273098,-153.80390649733803,-0.5529816881234011,167.79625505743851,-0.7084691109662841,-145.25051626584144,-0.7118603617623753,32.07426263909357,16.667652173570612,48.720400232778,0.3143269627403996,42.202451699575334,-2.942842294502287,0.6364663686348685,-0.037231810249444865,-0.0022936884919896147,-0.009301186672024907,0.000367657146109042

We can compare the calculated ephemerides to ephemerides provided by JPL Horizons to check their accuracy by calculating the maximum absolute residuals in RA, Dec, and heliocentric distance (r):

In [7]:
import numpy as np
eph_horizons = Ephem.from_horizons(['12893', '3552', '2018 RC3'], epochs=epochs, location='G37')

print('RA max residual:', np.max(np.fabs(eph['RA']-eph_horizons['RA']).to('arcsec')))
print('DEC max residual', np.max(np.fabs(eph['DEC']-eph_horizons['DEC']).to('arcsec')))
print('r max residual:', np.max(np.fabs(eph['r']-eph_horizons['r']).to('km')))

RA max residual: 0.018109424104295613 arcsec
DEC max residual 0.0180026742633288 arcsec
r max residual: 21.310798951714332 km


Querying Ephemerides from the Minor Planet Center and Miriade
======================

Ephemerides from the [Minor Planet Center](http://minorplanetcenter.net) can be queried using the function [sbpy.data.Ephem.from_mpc()](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.Ephem.html#sbpy.data.Ephem.from_mpc), which uses [astroquery.mpc](https://astroquery.readthedocs.io/en/latest/mpc/mpc.html) under the hood and uses the same syntax as [sbpy.data.Ephem.from_horizons](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.Ephem.html#sbpy.data.Ephem.from_horizons). Similarly, [sbpy.data.Ephem.from_miriade](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.Ephem.html#sbpy.data.Ephem.from_miriade) can be used to query ephemerides generated by the [Miriade system at IMCCE](http://vo.imcce.fr/webservices/miriade/), which uses [astroquery.imcce](https://astroquery.readthedocs.io/en/latest/imcce/imcce.html) under the hood. 

The following example compares ephemerides for asteroid Ceres for a given date and observer location from JPL Horizons and the Minor Planet Center:

In [8]:
from sbpy.data import Ephem
import numpy as np

epoch = Time(2451200.5, format='jd')
ceres_horizons = Ephem.from_horizons('Ceres', epochs=epoch, location=500)
ceres_mpc = Ephem.from_mpc('Ceres', epochs=epoch, location=500)
ceres_miriade = Ephem.from_miriade('Ceres', epochs=epoch, location=500)

In [9]:
# RA
print('Horizons', ceres_horizons['RA'])
print('MPC', ceres_horizons['RA'])
print('Miriade', ceres_miriade['RA'])

Horizons [57.37077] deg
MPC [57.37077] deg
Miriade [57.37067958] deg


In [10]:
# Dec
print('Horizons', ceres_horizons['DEC'])
print('MPC', ceres_horizons['DEC'])
print('Miriade', ceres_miriade['DEC'])

Horizons [19.50362] deg
MPC [19.50362] deg
Miriade [19.50358222] deg


In [11]:
# delta
print('Horizons', ceres_horizons['delta'])
print('MPC', ceres_horizons['delta'])
print('Miriade', ceres_miriade['delta'])

Horizons [2.09381638] AU
MPC [2.09381638] AU
Miriade [2.09381749] AU
